# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime
import numpy as np
tstart = '2017-10-23T00:00:00'
tend = '2017-10-24T00:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')

#tstart.strftime('%Y:%j:%H:%M:%S')


---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



../orbit_engine/lunar_296to297.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [3]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

#print(occfile)
orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2017-10-23 00:38:39,2017-10-23 01:35:02
1,2017-10-23 02:15:19,2017-10-23 03:11:41
2,2017-10-23 03:51:58,2017-10-23 04:48:21
3,2017-10-23 05:28:38,2017-10-23 06:25:00
4,2017-10-23 07:05:17,2017-10-23 08:01:39


In [4]:
#from nustar_planning.lunar_planning import position, position_shift
import nustar_planning.lunar_planning as lunar_planning
from importlib import reload
reload(lunar_planning)

<module 'nustar_planning.lunar_planning' from '/Users/bwgref/science/local/anaconda/lib/python3.5/site-packages/nustar_planning-0.1.dev50-py3.5.egg/nustar_planning/lunar_planning.py'>

In [6]:
reload(lunar_planning)
sub = orbits.iloc[5:6].reset_index()
lunar_planning.position(sub, steps = 5, parallax_correction=True)

Aim Time            RA         Dec
2017-10-23T08:41:57 250.26194  -17.23391
2017-10-23T08:53:13.400000 250.32637  -17.49258
2017-10-23T09:04:29.800000 249.89781  -17.64252
2017-10-23T09:15:46.200000 249.24470  -17.60822
2017-10-23T09:27:02.600000 248.76552  -17.41560


In [7]:
# PA == 0 (default), Rmoon = 940 arcseconds (default), 6 arcminutes maximum drift (default) and
# no minimum dwell period (default)
reload(lunar_planning)
lunar_planning.position_shift(sub, parallax_correction=True, pa = 338.0*u.deg, dt = 3*u.s,
                              outfile='lunar_20171023_v4.txt')

2017:296:08:36:57 RA: 250.26159  Dec: -17.40711

2017:296:08:39:03 RA: 250.34900  Dec: -17.46397

2017:296:08:41:27 RA: 250.42375  Dec: -17.53406

2017:296:08:44:21 RA: 250.46774  Dec: -17.62570

2017:296:08:48:12 RA: 250.43953  Dec: -17.72215

2017:296:08:52:36 RA: 250.35994  Dec: -17.78770

2017:296:08:56:06 RA: 250.26345  Dec: -17.83050

2017:296:08:58:54 RA: 250.16066  Dec: -17.85917

2017:296:09:01:18 RA: 250.05542  Dec: -17.87796

2017:296:09:03:27 RA: 249.94860  Dec: -17.88924

2017:296:09:05:27 RA: 249.84116  Dec: -17.89425

2017:296:09:07:21 RA: 249.73583  Dec: -17.89383

2017:296:09:09:09 RA: 249.62881  Dec: -17.88841

2017:296:09:10:57 RA: 249.52184  Dec: -17.87808

2017:296:09:12:45 RA: 249.41668  Dec: -17.86305

2017:296:09:14:33 RA: 249.31231  Dec: -17.84297

2017:296:09:16:24 RA: 249.20831  Dec: -17.81711

2017:296:09:18:21 RA: 249.10781  Dec: -17.78526

2017:296:09:20:24 RA: 249.01040  Dec: -17.74560

2017:296:09:22:39 RA: 248.91937  Dec: -17.69591

2017:296:09:25:12 RA

In [8]:
import pandas as pd

In [34]:
df3 = pd.read_csv('lunar_20171023_v3.txt', sep='\s+', skiprows=1, header=None, names=['Date', 'RA', 'Dec'])
df4 = pd.read_csv('lunar_20171023_v4.txt', sep='\s+', skiprows=1, header=None, names=['Date', 'RA', 'Dec'])



In [35]:
from astropy.coordinates import SkyCoord

In [39]:
skypos3 = SkyCoord(df3.RA, df3.Dec,  unit="deg")
skypos4 = SkyCoord(df4.RA, df4.Dec,  unit="deg")



In [46]:
for ind, pos3 in enumerate(skypos3):
    diff = pos3.separation(skypos4[ind])
    print(diff.arcsec)
    

0.535012935740757
0.3576610909146863
5.758718734996706
4.438753709270061
5.100031478860617
6.732669545164053
8.169955560004789
1.1584939633960178
1.2521897256066732
1.3481895171489073
1.3778854293156089
1.4429081673537671
1.4736041136139966
1.4394386307672513
1.4048456117935382
1.4068479017923265
1.3068584340864509
1.2424348313962044
1.118246645211824
7.382982554822452
5.721619927786878
8.167872207398336
9.842635408232349
6.816025429746975
8.366239856012221
